# Multiplexes generator

### General parameters

In [1]:
import math 
from datetime import datetime 
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [2]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)

In [3]:
####################################################
# Rewrite muxes
####################################################
# [MUX NAME, NUMBER OF INPUTS, INPUTS BIT WIDTH, DESCRIPTION, SIGNED/UNSIGED]
MUX_MATRIX = [['mux_sel_coord', 2, INPUT_COORD_WIDTH, 'Selects between the original coord (0) or the increased coord (1). For X and Y coords.', 'signed'],
              ['mux_sel_buffer_in', 2, OUT_2ND_INTERP_WIDTH, 'Selects between interpolation output (0) or external integer samples (1).', 'signed'],
              ['mux_sel_buffer_column', 8, OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, 'Selects the column to be read from the transposed buffer.', 'unsigned'],
              ['mux_sel_interp_line_in', 2, OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, 'Selects between the column from buffer (0) or external line (1).', 'unsigned'],
              ['mux_sel_dimension', 2, int(math.log2(FILTERS+1)), 'Selects between the MV_Y LSB (0) or the MV_X LSB (1).', 'unsigned'],
              ['mux_sel_filter', FILTERS+1, OUT_2ND_INTERP_WIDTH, 'Selects between the 15 outputs from interpolation.', 'signed']
              ]

In [4]:
###############################################
# Mux text template
###############################################
def generate_mux(ind):    
    # Modified date
    DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

    # Selector bit width
    SEL_WIDTH = int(math.log2(MUX_MATRIX[ind][1]))

    # Loop to generate the input names, sensitivity list and cases
    INPUT_NAME = []
    SENSIVITY_LIST = []
    CASE_LIST = []
    for i in range(MUX_MATRIX[ind][1]):
        # INPUT_NAME.append('  input ['+str(MUX_MATRIX[ind][2]-1)+':0] DATA_IN_'+str(i)+',')
        # SENSIVITY_LIST.append(' or DATA_IN_'+str(i))
        if ind != 2 and ind != 5:
            INPUT_NAME.append('  input '+MUX_MATRIX[ind][4]+' ['+str(MUX_MATRIX[ind][2]-1)+':0] DATA_IN_'+str(i)+',')
            SENSIVITY_LIST.append(' or DATA_IN_'+str(i))
            CASE_LIST.append('\t'+str(SEL_WIDTH)+"'b"+str(bin(i)[2:].zfill(SEL_WIDTH))+': DATA_OUT = DATA_IN_'+str(i)+';')
        elif ind == 2 and (i == 1 or i == 2 or i == 3 or i == 4):
            SENSIVITY_LIST.append(' or DATA_IN_'+str(i))
            INPUT_NAME.append('  input '+MUX_MATRIX[ind][4]+' ['+str(MUX_MATRIX[ind][2]-1)+':0] DATA_IN_'+str(i)+',')
            CASE_LIST.append('\t'+str(SEL_WIDTH)+"'b"+str(bin(i)[2:].zfill(SEL_WIDTH))+': DATA_OUT = DATA_IN_'+str(i)+';')
        elif ind == 2 and i == 0:
            CASE_LIST.append('\tdefault: DATA_OUT = '+str(MUX_MATRIX[ind][2])+"'b0;")
        elif ind == 5 and i != 15:
            INPUT_NAME.append('  input '+MUX_MATRIX[ind][4]+' ['+str(MUX_MATRIX[ind][2]-1)+':0] DATA_IN_'+str(i)+',')
            SENSIVITY_LIST.append(' or DATA_IN_'+str(i))
            CASE_LIST.append('\t'+str(SEL_WIDTH)+"'b"+str(bin(i)[2:].zfill(SEL_WIDTH))+': DATA_OUT = DATA_IN_'+str(i)+';')
        elif ind == 5 and i == 15:
            CASE_LIST.append('\tdefault: DATA_OUT = '+str(MUX_MATRIX[ind][2])+"'b0;")

    INPUT_NAME_CONCAT = '\n'.join(INPUT_NAME)
    SENSIVITY_LIST_CONCAT = ''.join(SENSIVITY_LIST)
    CASE_LIST_CONCAT = '\n'.join(CASE_LIST)

    # Register file text
    mux_verilog = '''/*-----------------------------------------------------------------------------------
* File: '''+MUX_MATRIX[ind][0]+'''.v
* Date generated: 25/03/2023
* Date modified: '''+DATE_MODIF+'''
* Author: Bruna Suemi Nagai
* Description: '''+str(MUX_MATRIX[ind][3])+'''
*----------------------------------------------------------------------------------- */

module '''+MUX_MATRIX[ind][0]+''' (
'''+INPUT_NAME_CONCAT+'''
  input ['''+str(SEL_WIDTH-1)+''':0] SELECT,
  output reg '''+MUX_MATRIX[ind][4]+''' ['''+str(MUX_MATRIX[ind][2]-1)+''':0] DATA_OUT
);
    

// ------------------------------------------
// Sequential logic
// ------------------------------------------
always @ (SELECT'''+SENSIVITY_LIST_CONCAT+''')
  case (SELECT)
'''+CASE_LIST_CONCAT+'''
  endcase

endmodule // '''+MUX_MATRIX[ind][0]+'''
    '''

    # Write the register file
    with open('../verilog/muxes/'+MUX_MATRIX[ind][0]+'.v', 'w') as mux_file:
        mux_file.write(mux_verilog)


In [5]:
# Define the list of words to select from
mux_names = ['Select a mux...', 'mux_sel_coord', 'mux_sel_buffer_in', 'mux_sel_buffer_column', 'mux_sel_interp_line_in', 
             'mux_sel_dimension', 'mux_sel_filter', 'Generate all']

# Define the initial values 
SELECTED_MUX = 'MUX_NOOB'
MUX_INDEX = 0

# Create the dropdown widget
word_select = widgets.Dropdown(
    options=mux_names,
    value=mux_names[0],
    description='Mux:')

# Create the button widget
button = widgets.Button(description='Generate mux file')

# Define a function to update the SELECTED_MUX variable
def update_mux_name(b):
    global MUX_INDEX
    SELECTED_MUX = word_select.value
    if SELECTED_MUX == 'mux_sel_coord':
        MUX_INDEX = 0
    elif SELECTED_MUX == 'mux_sel_buffer_in':
        MUX_INDEX = 1
    elif SELECTED_MUX == 'mux_sel_buffer_column':
        MUX_INDEX = 2
    elif SELECTED_MUX == 'mux_sel_interp_line_in':
        MUX_INDEX = 3
    elif SELECTED_MUX == 'mux_sel_dimension':
        MUX_INDEX = 4
    elif SELECTED_MUX == 'mux_sel_filter':
        MUX_INDEX = 5
    elif SELECTED_MUX == 'Generate all':
        MUX_INDEX = 6
    else:
        MUX_INDEX = 7

    # Loop for generate all    
    if MUX_INDEX == 6:
        for i in range(MUX_INDEX):
            generate_mux(i)
            print('Mux ', MUX_MATRIX[i][0], ' generated.')
            print('Index =', i)
        print('Finished generating all muxes :)')
    else:
        print('Mux', SELECTED_MUX, 'generated.')
        print('Index =', MUX_INDEX)
        generate_mux(MUX_INDEX)

# Attach the function to the button's on-click event
button.on_click(update_mux_name)

# Display the widgets
display(word_select)
display(button)


Dropdown(description='Mux:', options=('Select a mux...', 'mux_sel_coord', 'mux_sel_buffer_in', 'mux_sel_buffer…

Button(description='Generate mux file', style=ButtonStyle())

Mux mux_sel_buffer_in generated.
Index = 1
